In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
# seaborn for making statistical graphics in Python
import seaborn as sns 

from collections import Counter

#StandardScalre - Standardize features by removing the mean and scaling to unit variance
# LabelEncoder - converts each string value into a number
from sklearn.preprocessing import StandardScaler, LabelEncoder
# metrics implements functions assessing prediction error for specific purposes.
# includes score functions, performance metrics and pairwise metrics and distance computations.
from sklearn import metrics

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# # XGBoost - extreme gradient boosting
# import xgboost as xgb
# from xgboost import plot_importance
# # Light Gradient Boosting Machine - based on decision tree algorithms and used for ranking,
# # classification and other machine learning tasks
# import lightgbm as lgb
# from catboost import Pool, CatBoostRegressor

# for computing
from scipy.stats import uniform, randint
# from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

###### Data pre-processing

In [3]:
# import the dataset

complete_condo_data = pd.read_csv('Rental-Contracts-Non-Landed-Housing-Dev - removedNA.csv')
complete_condo_data.columns

Index(['S/N', 'Building/Project Name', 'Street Name', 'Postal District',
       'Type', 'No. of Bedroom(for Non-Landed Only)', 'Monthly Gross Rent($)',
       'Floor Area (sq ft)', 'Lease Commencement Date'],
      dtype='object')

In [4]:
columns_to_drop = ['S/N', 'Type']
valid_condo_data = complete_condo_data.drop(columns = columns_to_drop)
valid_condo_data.head()

,Building/Project Name,Street Name,Postal District,No. of Bedroom(for Non-Landed Only),Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
0,EMERALD PARK,INDUS ROAD,3,2,3200,900 to 1000,Feb-21
1,V ON SHENTON,SHENTON WAY,1,1,3700,700 to 800,Feb-21
2,MARINA BAY RESIDENCES,MARINA BOULEVARD,1,2,5850,1100 to 1200,Feb-21
3,THE TRILINQ,JALAN LEMPENG,5,1,2250,500 to 600,Feb-21
4,PEOPLE'S PARK COMPLEX,PARK ROAD,1,1,1800,400 to 500,Feb-21


###### Modeling - Random Forest Generator [https://towardsdatascience.com/random-forest-in-python-24d0893d51c0]

In [5]:
# One-hot encoding of categorical variables, using pandas get_dummies
valid_condo_data_dummy = valid_condo_data.copy()
valid_condo_data_dummy = pd.get_dummies(valid_condo_data_dummy, columns = ['Building/Project Name'], drop_first = False)
valid_condo_data_dummy = pd.get_dummies(valid_condo_data_dummy, columns = ['Street Name'], drop_first = False)
valid_condo_data_dummy = pd.get_dummies(valid_condo_data_dummy, columns = ['Floor Area (sq ft)'], drop_first = False)
valid_condo_data_dummy.head()

Postal District  No. of Bedroom(for Non-Landed Only)  \
0                3                                    2   
1                1                                    1   
2                1                                    2   
3                5                                    1   
4                1                                    1   

   Monthly Gross Rent($) Lease Commencement Date  \
0                   3200                  Feb-21   
1                   3700                  Feb-21   
2                   5850                  Feb-21   
3                   2250                  Feb-21   
4                   1800                  Feb-21   

   Building/Project Name_# 1 LOFT  Building/Project Name_# 1 SUITES  \
0                               0                                 0   
1                               0                                 0   
2                               0                                 0   
3                               0                                 0   
4                               0                                 0   

   Building/Project Name_1 KING ALBERT PARK  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   Building/Project Name_1 MOULMEIN RISE  Building/Project Name_1 NASSIM  \
0                                      0                               0   
1                                      0                               0   
2                                      0                               0   
3                                      0                               0   
4                                      0                               0   

   Building/Project Name_10 SHELFORD  Building/Project Name_105 LORONG G  \
0                                  0                                   0   
1                                  0                                   0   
2                                  0                                   0   
3                                  0                                   0   
4                                  0                                   0   

   Building/Project Name_11 AMBER ROAD  \
0                                    0   
1                                    0   
2                                    0   
3                                    0   
4                                    0   

   Building/Project Name_111 EMERALD HILL  Building/Project Name_16 @ AMBER  \
0                                       0                                 0   
1                                       0                                 0   
2                                       0                                 0   
3                                       0                                 0   
4                                       0                                 0   

   Building/Project Name_18 SHELFORD  Building/Project Name_18 WOODSVILLE  \
0                                  0                                    0   
1                                  0                                    0   
2                                  0                                    0   
3                                  0                                    0   
4                                  0                                    0   

   Building/Project Name_183 LONGHAUS  Building/Project Name_1919  \
0                                   0                           0   
1                                   0                           0   
2                                   0                           0   
3                                   0                           0   
4                                   0                           0   

   Building/Project Name_2 RVG  Building/Project Name_21 ANDERSON  \
0                            0      

In [6]:
# Separate dependent(target or label) variable and independent (features) variables
# The target, also known as the label, is the value we want to predict and
# the features are all the columns the model uses to make a prediction. 
target = 'Monthly Gross Rent($)'
X = valid_condo_data_dummy.drop([target], axis = 1) # features
# Saving feature names for later use
X_list = list(X.columns)
y = valid_condo_data_dummy[target] # label

In [7]:
# Perform train test split - Splits data into training and testing sets.
# During training, let the model ‘see’ the answers, in this case the rent, so it can learn how to predict the rent from the features.
# Expecting that there is some relationship between all the features and the target value, 
# and the model’s job is to learn this relationship during training.
# Then, when it comes time to evaluate the model, we ask it to make predictions on a testing set where it only has access to the features 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)


In [8]:
# Create the parameter grid for GridSearchCV
rf_param_grid = {
    'max_depth': [80, 90, 100], # Maximum number of levels in each decision tree
    'max_features': [2, 3], # Maximum number of features considered for splitting a node
    'min_samples_leaf': [1, 3, 4, 5], # Minimum number of data points allowed in a leaf node
    'n_estimators': [100, 300, 600] # Number of trees in the forest
}

In [9]:
# Instantiate a Random Forest Regressor
rf_reg = RandomForestRegressor(random_state = 42)

In [10]:
# Setup grid search
rf_grid = GridSearchCV(estimator = rf_reg, param_grid = rf_param_grid, n_jobs = 1, cv=5, verbose=1,)
# Fit the grid search model on training dataset
grid_fit = rf_grid.fit(X_train, y_train)
bestParams = rf_grid.best_params_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


MemoryError: Unable to allocate 6.96 GiB for an array with shape (2797, 333891) and data type object

In [ ]:
# Get best RF estimator based on best parameters
rf_best = rf_grid.best_estimator_

# Use RF regressor to predict on test set
y_pred_test = rf_best.predict(X_test)


In [ ]:
# Store results as dataframe
rf_dict = {'Model':'Random Forest Regressor',
          'R^2':metrics.r2_score(y_test, y_pred_test),
          'Adjusted R^2':(1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)),
          'MAE':metrics.mean_absolute_error(y_test, y_pred_test),
          'MSE':metrics.mean_squared_error(y_test, y_pred_test),
          'RMSE':np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))}

rf_reg_metrics = pd.DataFrame.from_dict(rf_dict, orient = 'index').T

# Display model performance metrics
rf_reg_metrics

In [ ]:
rf_best.feature_importances_

In [ ]:
rf_features = pd.DataFrame([X_train.columns, rf_best.feature_importances_]).T
rf_features = rf_features.rename(columns={0: 'Feature', 1: 'Importance Score'})
rf_features.sort_values(by = 'Importance Score', ascending = False)